In [8]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this task you will build a classifier for Rock-Paper-Scissors 
# based on the rps dataset.
#
# IMPORTANT: Your final layer should be as shown, do not change the
# provided code, or the tests may fail
#
# IMPORTANT: Images will be tested as 150x150 with 3 bytes of color depth
# So ensure that your input layer is designed accordingly, or the tests
# may fail. 
#
# NOTE THAT THIS IS UNLABELLED DATA. 
# You can use the ImageDataGenerator to automatically label it
# and we have provided some starter code.

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
    urllib.request.urlretrieve(url, 'rps.zip')
    local_zip = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('./tmp/')
    zip_ref.close()


    TRAINING_DIR = "./tmp/rps/"
    training_datagen = ImageDataGenerator(
    # YOUR CODE HERE
    rescale=1/255.,
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True
    )

#     train_generator = # YOUR CODE HERE
    train_generator = training_datagen.flow_from_directory(
        './tmp/rps/',
        target_size=(150, 150),
        batch_size=32,
        class_mode = 'categorical'
    )
    
    valid_datagen = ImageDataGenerator(
        rescale=1/255.,
     )

    valid_generator = valid_datagen.flow_from_directory(
            TRAINING_DIR,
            target_size=(150, 150),
            batch_size=32,
            class_mode = 'categorical'
     )
    conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
    conv_base.trainable = False
    
    model = tf.keras.models.Sequential([
        conv_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),                                       

    # YOUR CODE HERE, BUT END WITH A 3 Neuron Dense, activated by softmax
        tf.keras.layers.Dense(3, activation='softmax')
    ])


    model.compile(loss='categorical_crossentropy', optimizer='adam', 
                 metrics=['acc'])
    checkpoint_path = "my_checkpoint_rps2.ckpt"
    checkpoint = ModelCheckpoint(filepath= checkpoint_path, save_weights_only=True,
                                save_best_only=True, monitor='val_loss', verbose=1)
    model.fit(train_generator, validation_data=(valid_generator), epochs=20
             ,callbacks=[checkpoint])
    model.load_weights(checkpoint_path)
    return model



# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("cat3_rps2_vgg.h5")

Found 2520 images belonging to 3 classes.
Found 2520 images belonging to 3 classes.
Epoch 1/20
79/79 [==============================] - ETA: 0s - loss: 0.1842 - acc: 0.9353
Epoch 00001: val_loss improved from inf to 0.00239, saving model to my_checkpoint_rps2.ckpt
79/79 [==============================] - 116s 1s/step - loss: 0.1842 - acc: 0.9353 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 2/20
79/79 [==============================] - ETA: 0s - loss: 0.0132 - acc: 0.9948
Epoch 00002: val_loss improved from 0.00239 to 0.00007, saving model to my_checkpoint_rps2.ckpt
79/79 [==============================] - 115s 1s/step - loss: 0.0132 - acc: 0.9948 - val_loss: 7.0918e-05 - val_acc: 1.0000
Epoch 3/20
79/79 [==============================] - ETA: 0s - loss: 0.0083 - acc: 0.9968
Epoch 00003: val_loss did not improve from 0.00007
79/79 [==============================] - 115s 1s/step - loss: 0.0083 - acc: 0.9968 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 4/20
79/79 [============================